In [1]:
%load_ext autoreload

In [2]:
import torch 
import numpy as np

from pixel_shuffle import pixel_shuffle, pixel_unshuffle

%autoreload 2

In [3]:
inputs = torch.randn(1, 64, 2, 3, 4)
factor = 4

In [4]:
print('inputs shape: ', inputs.size())
shuffled = pixel_shuffle(inputs, factor)
print('shuffled shape: ', shuffled.size())
unshuffled = pixel_unshuffle(shuffled, factor)
print('unshuffled shape: ', unshuffled.size())

assert np.all(unshuffled.numpy() == inputs.numpy())

inputs shape:  torch.Size([1, 64, 2, 3, 4])
shuffled shape:  torch.Size([1, 1, 8, 12, 16])
unshuffled shape:  torch.Size([1, 64, 2, 3, 4])
